# Cost Analysis of ML Model Generations

## Objective
Calculate the cost of generations for different ML services based on:
- Compute node runtime (A100 and L4)
- Scheduled node activation/deactivation (cron-based)
- **Actual execution time** of each task (from 'running' status until completion)
- **Separate cost allocation** for successful and failed tasks
- Proportional distribution of costs by usage time

## Key Principles
1. **Node schedule**: each record = timestamp with the number of active nodes
2. **Node types**: A100 (more expensive) and L4 (cheaper), determined by node pool name
3. **Execution time**: measured from 'running' status to any terminal status
4. **Separate allocation rules**:
   - Successful tasks: cost per task
   - Failed tasks: cost per second of execution
5. **Cost distribution**: proportional to node usage time
6. **Analysis period**: June 1 – August 31
7. **Focus on ML services**: API-based services excluded for data consistency


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
from sqlalchemy import create_engine, text
# from config import PG_DATABASE_URL # .env credentials for security
from tqdm import tqdm

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')
sns.set_palette("husl")

# Database connection
engine = create_engine(PG_DATABASE_URL)
print("Database connection established")

# Hourly node pricing
PRICES = {
    'a100': 11,  # $/hour
    'l4': 1   # $/hour
}
print(f"Pricing: A100 ${PRICES['a100']}/hour, L4 ${PRICES['l4']}/hour")


Database connection established
Pricing: A100 $11/hour, L4 $1/hour


## Step 1: Node Schedule Analysis

We load node schedule data from the `k8s_nodes_scaling` table.
Each record contains:
- `time_cron`: time in cron format (hour:minute)
- `nodes_count`: number of active nodes from this point in time
- `is_weekend`: weekend flag (true/false)
- `node_pool_name`: name of the node pool (defines type: A100 or L4)

In [ ]:
# Load node schedule data
query_schedule = open("sql/node_schedule_analysis.sql").read()
df_schedule_raw = pd.read_sql(query_schedule, engine)
print(f"{len(df_schedule_raw)} schedule entries loaded")

# Show data structure
print("\nSchedule data structure:")
display(df_schedule_raw.head())

print("\nUnique values:")
print(f"Node pool types: {df_schedule_raw['node_pool_name'].unique()}")
print(f"Weekend flag: {df_schedule_raw['is_weekend'].unique()}")
print(f"Sample cron entries: {df_schedule_raw['time_cron'].head().tolist()}")

**Mock output (illustrative example):**

Schedule data structure (head):

| id | node_pool_name | nodes_count | is_weekend | time_cron     | creation_ts          |
|----|----------------|-------------|------------|---------------|----------------------|
| 1  | a100           | 2           | False      | `0 8 * * 1-5` | 2025-06-01 08:00:00  |
| 2  | a100           | 4           | False      | `0 12 * * 1-5`| 2025-06-01 12:00:00  |
| 3  | l4             | 6           | False      | `0 9 * * 1-5` | 2025-06-01 09:00:00  |
| 4  | l4             | 12          | True       | `0 15 * * 0,6`| 2025-06-02 15:00:00  |
| 5  | a100           | 0           | True       | `0 0 * * 0,6` | 2025-06-02 00:00:00  |

Unique values:
- Node pool types: `['a100', 'l4']`
- Weekend flag: `[False, True]`
- Sample cron entries: `['0 8 * * 1-5', '0 12 * * 1-5', '0 9 * * 1-5', '0 15 * * 0,6', '0 0 * * 0,6']`


## Step 2: Parsing cron expressions and building a continuous schedule

1. **Parse cron expressions** to extract hours and minutes
2. **Classify node types** based on pool names
3. **Build a continuous 24-hour schedule** for each day type (weekday / weekend)
4. **Calculate node-hours** per day for each node type

**Logic:** Fill all intermediate hours when a node was active, up until the schedule changes to the next state.

In [ ]:
# Parse cron schedule
def parse_cron_schedule(df_schedule_raw):
    parsed_schedules = []

    for _, row in df_schedule_raw.iterrows():
        cron_parts = row['time_cron'].split(' ')

        if len(cron_parts) >= 5:
            minute = cron_parts[0]
            hour = cron_parts[1]

            # Determine node type by pool name
            if 'a100' in row['node_pool_name'].lower():
                node_type = 'a100'
            else:
                node_type = 'l4'

            try:
                hour_int = int(hour) if hour != '*' else 0
                minute_int = int(minute) if minute != '*' else 0

                parsed_schedules.append({
                    'id': row['id'],
                    'node_pool_name': row['node_pool_name'],
                    'node_type': node_type,
                    'nodes_count': row['nodes_count'],
                    'is_weekend': row['is_weekend'],
                    'time_cron': row['time_cron'],
                    'hour_int': hour_int,
                    'minute_int': minute_int,
                    'creation_ts': row['creation_ts']
                })
            except ValueError:
                print(f"Skipping invalid cron: {row['time_cron']}")

    df_parsed = pd.DataFrame(parsed_schedules)
    # Sort by numeric values for correct order
    df_parsed = df_parsed.sort_values(['is_weekend', 'node_type', 'hour_int', 'minute_int'])

    return df_parsed

# Parse the schedule
df_schedule_parsed = parse_cron_schedule(df_schedule_raw)
print(f"Parsing completed: {len(df_schedule_parsed)} records")

if len(df_schedule_parsed) > 0:
    print("\nFirst 20 parsed records (sorted):")
    display(df_schedule_parsed.head(20))

**Mock output (illustrative example):**

First 5 parsed records:

| id | node_pool_name | node_type | nodes_count | is_weekend | time_cron     | hour_int | minute_int | creation_ts          |
|----|----------------|-----------|-------------|------------|---------------|----------|------------|----------------------|
| 1  | pool-a100      | a100      | 2           | False      | `0 8 * * 1-5` | 8        | 0          | 2025-06-01 08:00:00  |
| 2  | pool-a100      | a100      | 4           | False      | `0 12 * * 1-5`| 12       | 0          | 2025-06-01 12:00:00  |
| 3  | pool-l4        | l4        | 6           | False      | `0 9 * * 1-5` | 9        | 0          | 2025-06-01 09:00:00  |
| 4  | pool-l4        | l4        | 12          | True       | `0 15 * * 0,6`| 15       | 0          | 2025-06-02 15:00:00  |
| 5  | pool-a100      | a100      | 0           | True       | `0 0 * * 0,6` | 0        | 0          | 2025-06-02 00:00:00  |


## Step 3: Building a continuous 24-hour schedule

We fill in all intermediate hours when a node remained active, until the next schedule change occurs.
This provides a complete picture of node utilization throughout the day.

**Key idea:** If a node was turned on at 07:00 and the next change is at 09:00,
then it was also active at 08:00.

In [ ]:
# Create a continuous schedule by filling all intermediate hours
def create_continuous_schedule(df_schedule_parsed):
    continuous_schedules = []

    for is_weekend in [False, True]:
        for node_type in ['a100', 'l4']:
            subset = df_schedule_parsed[(df_schedule_parsed['is_weekend'] == is_weekend) &
                                        (df_schedule_parsed['node_type'] == node_type)]

            if len(subset) > 0:
                # Sort by hours for correct ordering
                subset = subset.sort_values('hour_int')

                # Build continuous schedule for all 24 hours
                current_nodes = 0

                for hour in range(24):
                    # Check if there is a record for this hour
                    hour_record = subset[subset['hour_int'] == hour]

                    if len(hour_record) > 0:
                        # Update active node count
                        current_nodes = hour_record.iloc[0]['nodes_count']

                    # Add record for this hour (even if no change happened)
                    continuous_schedules.append({
                        'is_weekend': is_weekend,
                        'node_type': node_type,
                        'hour': hour,
                        'nodes_count': current_nodes,
                        'is_schedule_change': len(hour_record) > 0
                    })

    return pd.DataFrame(continuous_schedules)

# Build continuous schedule
df_continuous = create_continuous_schedule(df_schedule_parsed)
print(f"Continuous schedule created: {len(df_continuous)} records")
print("Expected: 24 hours × 2 day types × 2 node types = 96 records")

if len(df_continuous) > 0:
    print("\n=== HOURLY STATS ===")
    for is_weekend in [False, True]:
        for node_type in ['a100', 'l4']:
            subset = df_continuous[(df_continuous['is_weekend'] == is_weekend) &
                                   (df_continuous['node_type'] == node_type)]

            if len(subset) > 0:
                total_hours = len(subset)
                working_hours = len(subset[subset['nodes_count'] > 0])
                idle_hours = total_hours - working_hours

                print(f"\n{node_type} (weekend? {is_weekend}):")
                print(f"  Total hours: {total_hours}")
                print(f"  Active hours: {working_hours}")
                print(f"  Idle hours: {idle_hours}")

                # Show working hours
                working_schedule = subset[subset['nodes_count'] > 0]
                if len(working_schedule) > 0:
                    working_hours_list = working_schedule['hour'].tolist()
                    print(f"  Active at hours: {working_hours_list}")

**Mock output (illustrative example):**

Continuous schedule created: 96 records
Expected: 24 hours × 2 day types × 2 node types = 96 records

=== HOURLY STATS ===

a100 (weekend? False):
- Total hours: 24
- Active hours: 10
- Idle hours: 14
- Active at hours: [8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

l4 (weekend? False):
- Total hours: 24
- Active hours: 12
- Idle hours: 12
- Active at hours: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

a100 (weekend? True):
- Total hours: 24
- Active hours: 6
- Idle hours: 18
- Active at hours: [10, 11, 12, 13, 14, 15]

l4 (weekend? True):
- Total hours: 24
- Active hours: 8
- Idle hours: 16
- Active at hours: [12, 13, 14, 15, 16, 17, 18, 19]

## Step 4: Daily node-hours calculation

We calculate the total number of node-hours per day for each node type and day type.
This value is later used to estimate the daily cost of node usage.

**Formula:** sum of node counts across all hours of the day.

In [ ]:
# Calculate total node-hours per day for each node type and day type
def calculate_daily_node_hours(df_continuous):
    daily_hours = []

    for is_weekend in [False, True]:
        for node_type in ['a100', 'l4']:
            subset = df_continuous[(df_continuous['is_weekend'] == is_weekend) &
                                   (df_continuous['node_type'] == node_type)]

            if len(subset) > 0:
                # Total node-hours per day = sum of active nodes across all hours
                total_node_hours_per_day = subset['nodes_count'].sum()

                daily_hours.append({
                    'is_weekend': is_weekend,
                    'node_type': node_type,
                    'total_node_hours_per_day': total_node_hours_per_day,
                    'working_hours': len(subset[subset['nodes_count'] > 0])
                })

    return pd.DataFrame(daily_hours)

# Calculate node-hours per day
df_daily_hours = calculate_daily_node_hours(df_continuous)
print("Daily node-hours calculated!")

if len(df_daily_hours) > 0:
    print("\n=== DAILY NODE-HOURS ===")
    display(df_daily_hours)

**Mock output (illustrative example):**

Daily node-hours per day:

| is_weekend | node_type | total_node_hours_per_day | working_hours |
|------------|-----------|--------------------------|---------------|
| False      | a100      | 20                       | 10            |
| False      | l4        | 36                       | 12            |
| True       | a100      | 12                       | 6             |
| True       | l4        | 16                       | 8             |

## Step 5: Daily node cost calculation

We calculate the daily cost of node usage starting from June 1, 2025,
taking into account the day type (weekday/weekend) and the corresponding number of node-hours.

**Formula:** Node-hours × Hourly price

In [ ]:
# Calculate daily node costs from June 1
def calculate_daily_costs_from_june(df_daily_hours):
    start_date = datetime(2025, 6, 1)
    end_date = datetime(2025, 8, 21)

    daily_costs = []
    current_date = start_date

    print("Calculating daily costs...")
    total_days = (end_date - start_date).days + 1

    with tqdm(total=total_days, desc="Daily cost calculation") as pbar:
        while current_date <= end_date:
            # Determine if current day is a weekend
            is_weekend = current_date.weekday() in [5, 6]  # 5=Saturday, 6=Sunday

            # Find corresponding node-hours
            for _, row in df_daily_hours.iterrows():
                if row['is_weekend'] == is_weekend:
                    node_type = row['node_type']
                    node_hours = row['total_node_hours_per_day']
                    hourly_price = PRICES[node_type]
                    daily_cost = node_hours * hourly_price

                    daily_costs.append({
                        'date': current_date.date(),
                        'is_weekend': is_weekend,
                        'node_type': node_type,
                        'node_hours': node_hours,
                        'hourly_price': hourly_price,
                        'daily_cost': daily_cost
                    })

            current_date += timedelta(days=1)
            pbar.update(1)

    return pd.DataFrame(daily_costs)

# Calculate daily costs starting June 1
df_daily_costs = calculate_daily_costs_from_june(df_daily_hours)

print(f"Daily costs calculated: {len(df_daily_costs)} records")

if len(df_daily_costs) > 0:
    print("\n=== SUMMARY STATISTICS ===")
    total_cost_by_node = df_daily_costs.groupby('node_type')['daily_cost'].sum()
    print("Total cost by node type:")
    for node_type, total_cost in total_cost_by_node.items():
        print(f"  {node_type}: ${total_cost:.2f}")

    total_cost = total_cost_by_node.sum()
    print(f"\nTotal cost (all nodes): ${total_cost:.2f}")

    # Show first 10 records
    print("\n=== FIRST 10 DAYS ===")
    display(df_daily_costs.head(10))

**Mock output (illustrative example):**

Daily costs calculated: 168 records

=== SUMMARY STATISTICS ===
Total cost by node type:
- a100: $6,600.00
- l4: $2,800.00

Total cost (all nodes): $9,400.00

=== FIRST 10 DAYS ===

| date       | is_weekend | node_type | node_hours | hourly_price | daily_cost |
|------------|------------|-----------|------------|--------------|------------|
| 2025-06-01 | False      | a100      | 20         | 11.00        | 220.00     |
| 2025-06-01 | False      | l4        | 36         | 1.00         | 36.00      |
| 2025-06-02 | False      | a100      | 20         | 11.00        | 220.00     |
| 2025-06-02 | False      | l4        | 36         | 1.00         | 36.00      |
| 2025-06-03 | False      | a100      | 20         | 11.00        | 220.00     |
| 2025-06-03 | False      | l4        | 36         | 1.00         | 36.00      |
| 2025-06-04 | False      | a100      | 20         | 11.00        | 220.00     |
| 2025-06-04 | False      | l4        | 36         | 1.00         | 36.00      |
| 2025-06-05 | False      | a100      | 20         | 11.00        | 220.00     |
| 2025-06-05 | False      | l4        | 36         | 1.00         | 36.00      |

## Step 6: Task analysis and execution duration calculation

**Key points:**
1. **Load task data** from June 1 to August 31 (all statuses except: new, waiting, pending)
2. **Exclude API services** to keep analysis focused on ML nodes
3. **Extract execution history** from the `task_history` table
4. **Calculate execution time** from status `running` to any terminal status
5. **Separate successful vs. failed tasks** for different cost allocation (per-task vs. per-second)
6. **Classify node types** based on `metainfo`

**Terminal statuses:** `done`, `cancelled`, `execution_timeout`, `error_params`, `error_node`

In [ ]:
# Load task history for execution duration calculation
query_task_history = open("sql/task_data.sql").read()

df_task_history = pd.read_sql(query_task_history, engine)
print(f"Task history loaded: {len(df_task_history)} records")

# Show data structure
print("\nTask history structure:")
display(df_task_history.head())

print("\nUnique statuses:")
print(df_task_history['status'].unique())

print("\nUnique services:")
print(df_task_history['service_name'].unique())

**Mock output (illustrative example):**

Task history structure (head):

| task_id | service_name | status       | start_time          | end_time            | node_type | user_id |
|---------|--------------|--------------|---------------------|---------------------|-----------|---------|
| 101     | gen_image    | running      | 2025-06-01 08:01:00 | 2025-06-01 08:01:05 | a100      | u001    |
| 102     | gen_image    | done         | 2025-06-01 08:02:00 | 2025-06-01 08:02:30 | l4        | u002    |
| 103     | upscale      | error_node   | 2025-06-01 08:05:00 | 2025-06-01 08:05:10 | a100      | u003    |
| 104     | gen_text     | done         | 2025-06-01 08:10:00 | 2025-06-01 08:10:20 | l4        | u004    |
| 105     | gen_video    | cancelled    | 2025-06-01 08:15:00 | 2025-06-01 08:15:25 | a100      | u005    |

Unique statuses:
`['running', 'done', 'cancelled', 'execution_timeout', 'error_params', 'error_node']`

Unique services:
`['gen_image', 'gen_text', 'gen_video', 'upscale']`

## Step 7: Extracting task execution intervals

**Core logic:** extract intervals from status `running` to a terminal status.
This gives the actual execution time of a task, excluding time spent in queue.

**Advantages of this approach:**
- Only real resource usage is counted
- Separate successful vs. failed tasks
- Accurate execution duration is obtained

In [ ]:
# Function: extract execution intervals (from running to terminal status)
def extract_run_intervals(df_task_history):
    intervals = []
    terminal_statuses = {'done', 'error', 'error_cuda', 'canceled', 'error_exec_timeout', 'error_params'}

    with tqdm(total=df_task_history.task_id.nunique()) as progressbar:
        for task_id, group in df_task_history.groupby("task_id"):
            group = group.sort_values("creation_ts")
            running_rows = group[group.status == "running"]

            if running_rows.empty:
                continue

            start_time = running_rows.iloc[0].creation_ts
            is_weekend = running_rows.iloc[0].is_weekend

            # Take all events after running
            after_running = group[group.creation_ts > start_time]

            if after_running.empty:
                # No completion → skip task
                continue

            end_row = after_running.iloc[-1]
            end_time = end_row.creation_ts
            status = end_row.status

            if status not in terminal_statuses:
                # If last status is not terminal → skip task
                continue

            intervals.append({
                "task_id": task_id,
                "service_name": group.service_name.iloc[0],
                "node_id": group.node_id.iloc[0],
                "metainfo": group.metainfo.iloc[0],
                "start": start_time,
                "end": end_time,
                "is_weekend": is_weekend,
                "duration_seconds": (end_time - start_time).total_seconds(),
                "success": status == "done"
            })

    return pd.DataFrame(intervals)

# Extract execution intervals
df_intervals = extract_run_intervals(df_task_history)
print(f"Execution intervals extracted for {len(df_intervals)} tasks")

if len(df_intervals) > 0:
    print("\n=== INTERVAL STATISTICS ===")
    print(f"Total duration across all tasks: {df_intervals['duration_seconds'].sum():,.0f} seconds")
    print(f"Average task duration: {df_intervals['duration_seconds'].mean():.1f} seconds")
    print(f"Median task duration: {df_intervals['duration_seconds'].median():.1f} seconds")

    # Success rate stats
    success_count = df_intervals['success'].sum()
    total_count = len(df_intervals)
    print(f"\nSuccessful tasks: {success_count} ({success_count/total_count*100:.1f}%)")
    print(f"Failed tasks: {total_count - success_count} ({(total_count-success_count)/total_count*100:.1f}%)")

    print("\n=== FIRST 10 INTERVALS ===")
    display(df_intervals.head(10))

**Mock output (illustrative example):**

Execution intervals extracted for 500 tasks

=== INTERVAL STATISTICS ===
Total duration across all tasks: 25,000 seconds
Average task duration: 50.0 seconds
Median task duration: 30.0 seconds

Successful tasks: 420 (84.0%)
Failed tasks: 80 (16.0%)

=== FIRST 10 INTERVALS ===

| task_id | service_name | node_id | metainfo | start               | end                 | is_weekend | duration_seconds | success |
|---------|--------------|---------|----------|---------------------|---------------------|------------|------------------|---------|
| 201     | gen_image    | n1      | {...}    | 2025-06-01 08:00:00 | 2025-06-01 08:00:30 | False      | 30               | True    |
| 202     | gen_text     | n2      | {...}    | 2025-06-01 08:05:00 | 2025-06-01 08:05:45 | False      | 45               | True    |
| 203     | upscale      | n3      | {...}    | 2025-06-01 08:10:00 | 2025-06-01 08:10:15 | False      | 15               | False   |
| 204     | gen_image    | n1      | {...}    | 2025-06-01 08:20:00 | 2025-06-01 08:21:10 | False      | 70               | True    |
| 205     | gen_video    | n2      | {...}    | 2025-06-01 08:30:00 | 2025-06-01 08:31:00 | False      | 60               | True    |
| 206     | gen_text     | n3      | {...}    | 2025-06-01 08:40:00 | 2025-06-01 08:40:20 | False      | 20               | False   |
| 207     | gen_image    | n1      | {...}    | 2025-06-01 08:50:00 | 2025-06-01 08:50:55 | False      | 55               | True    |
| 208     | upscale      | n3      | {...}    | 2025-06-01 09:00:00 | 2025-06-01 09:00:35 | False      | 35               | True    |
| 209     | gen_image    | n2      | {...}    | 2025-06-01 09:10:00 | 2025-06-01 09:10:25 | False      | 15               | False   |
| 210     | gen_text     | n1      | {...}    | 2025-06-01 09:20:00 | 2025-06-01 09:21:05 | False      | 65               | True    |

## Step 8: Node classification and daily grouping

1. **Determine node type** (A100/L4) from `metainfo.node_name`
2. **Add date** for grouping
3. **Group by day, service, and node type** with duration taken into account
4. **Separate successful vs. failed tasks** for different cost allocation

**Grouping logic:**
- Successful tasks: count and total duration
- Failed tasks: count and total duration

In [ ]:
# Node classification and grouping of successful/failed tasks

# Determine node type from metainfo
def classify_node_type(metainfo):
    try:
        if isinstance(metainfo, str):
            metainfo = json.loads(metainfo)

        node_name = metainfo.get('node_name', '').lower()

        if 'a100' in node_name:
            return 'a100'
        else:
            return 'l4'
    except:
        return 'l4'  # Default fallback

# Add node classification
df_intervals["node_type"] = df_intervals.metainfo.apply(classify_node_type)
df_intervals["date"] = df_intervals.start.dt.date

print("Node classification completed")

# Group by day, service, and node type, considering duration
df_daily_tasks = (
    df_intervals
    .groupby(["date","start","is_weekend","task_id","service_name","node_type","success"])
    .agg(task_count=("task_id","nunique"), total_seconds=("duration_seconds","sum"))
    .reset_index()
)

print(f"Daily grouping completed: {len(df_daily_tasks)} records")

# Show statistics
print("\n=== NODE TYPE STATISTICS ===")
node_stats = df_intervals.groupby('node_type').agg({
    'task_id': 'count',
    'duration_seconds': ['sum', 'mean']
})
node_stats.columns = ['task_count', 'total_duration', 'avg_duration']
print(node_stats)

print("\n=== WEEKDAY / WEEKEND STATISTICS ===")
weekday_stats = df_intervals.groupby('is_weekend').agg({
    'task_id': 'count',
    'duration_seconds': ['sum', 'mean']
})
weekday_stats.columns = ['task_count', 'total_duration', 'avg_duration']
print(weekday_stats)

print("\n=== FIRST 15 GROUPED RECORDS ===")
display(df_daily_tasks.head(15))

**Mock output (illustrative example):**

Node classification completed
Daily grouping completed: 1,200 records

=== NODE TYPE STATISTICS ===

| node_type | task_count | total_duration | avg_duration |
|-----------|------------|----------------|--------------|
| a100      | 700        | 35,000         | 50.0         |
| l4        | 500        | 12,000         | 24.0         |

=== WEEKDAY / WEEKEND STATISTICS ===

| is_weekend | task_count | total_duration | avg_duration |
|------------|------------|----------------|--------------|
| False      | 900        | 32,000         | 35.6         |
| True       | 300        | 15,000         | 50.0         |

=== FIRST 15 GROUPED RECORDS ===

| date       | start               | is_weekend | task_id | service_name | node_type | success | task_count | total_seconds |
|------------|---------------------|------------|---------|--------------|-----------|---------|------------|---------------|
| 2025-06-01 | 2025-06-01 08:00:00 | False      | 201     | gen_image    | a100      | True    | 1          | 30            |
| 2025-06-01 | 2025-06-01 08:05:00 | False      | 202     | gen_text     | l4        | True    | 1          | 45            |
| 2025-06-01 | 2025-06-01 08:10:00 | False      | 203     | upscale      | a100      | False   | 1          | 15            |
| 2025-06-01 | 2025-06-01 08:20:00 | False      | 204     | gen_image    | a100      | True    | 1          | 70            |
| 2025-06-01 | 2025-06-01 08:30:00 | False      | 205     | gen_video    | l4        | True    | 1          | 60            |
| ...        | ...                 | ...        | ...     | ...          | ...       | ...     | ...        | ...           |

## Step 9: Cost allocation proportional to execution time

**Core logic:** Node costs are not distributed equally across services,
but proportionally to the execution time of their tasks.

**Separate allocation rules:**
- **Successful tasks**: cost per ONE generation = allocated cost / number of tasks
- **Failed tasks**: cost per SECOND = allocated cost / total execution time

**Allocation formula:**
1. For each day and node type: total node cost per day
2. Proportional allocation: (service time / total time) × daily node cost
3. Tariff: successful → per task, failed → per second

In [ ]:
# Allocate node costs proportionally to execution time of tasks
def calculate_cost_per_generation_by_duration(df_daily_tasks, df_daily_costs):
    # Create a working copy
    df_result = df_daily_tasks.copy()

    # Merge with daily node costs
    merged = df_result.merge(
        df_daily_costs[['date', 'node_type', 'is_weekend', 'daily_cost']],
        on=['date', 'node_type', 'is_weekend'],
        how='left'
    )

    # Group by day and node type to calculate total duration
    daily_durations = df_result.groupby(['date', 'node_type'])['total_seconds'].sum().reset_index()
    daily_durations.columns = ['date', 'node_type', 'total_duration_day_node']

    # Merge with main table
    result = merged.merge(daily_durations, on=['date', 'node_type'], how='left')

    # Vectorized calculations
    result['duration_ratio'] = result['total_seconds'] / result['total_duration_day_node']
    result['allocated_cost'] = result['daily_cost'] * result['duration_ratio']

    # Separate pricing rules
    result['cost_per_generation'] = np.where(
        result['success'],
        result['allocated_cost'] / result['task_count'].replace(0, 1),
        np.nan
    )

    result['cost_per_second'] = np.where(
        ~result['success'],
        result['allocated_cost'] / result['total_seconds'].replace(0, 1),
        np.nan
    )

    # Rename column
    result = result.rename(columns={'daily_cost': 'daily_node_cost'})

    # Update the original dataframe
    df_result['duration_ratio'] = result['duration_ratio']
    df_result['daily_node_cost'] = result['daily_node_cost']
    df_result['allocated_cost'] = result['allocated_cost']
    df_result['cost_per_generation'] = result['cost_per_generation']
    df_result['cost_per_second'] = result['cost_per_second']

    return df_result

# Calculate cost per generation
df_cost_per_gen_daily = calculate_cost_per_generation_by_duration(
    df_daily_tasks, df_daily_costs
)

print(f"Cost per generation calculated: {len(df_cost_per_gen_daily)} records")

if len(df_cost_per_gen_daily) > 0:
    print("\n=== CHECK CALCULATIONS ===")

    # Total number of tasks
    total_tasks = df_cost_per_gen_daily.task_id.nunique()
    print(f"Total tasks: {total_tasks:,}")

    # Total allocated cost
    total_allocated_cost = df_cost_per_gen_daily['allocated_cost'].sum()
    print(f"Total allocated cost: ${total_allocated_cost:.2f}")

    # Stats by success/failure
    success_tasks = df_cost_per_gen_daily[df_cost_per_gen_daily['success']==True]
    fail_tasks = df_cost_per_gen_daily[df_cost_per_gen_daily['success']==False]

    print(f"\nSuccessful tasks: {success_tasks.task_count.sum()}")
    print(f"Failed tasks: {fail_tasks.task_count.sum()}")

    print("\n=== FIRST 10 RECORDS ===")
    example = df_cost_per_gen_daily.head(10)
    display(example)

**Mock output (illustrative example):**

Cost per generation calculated: 1,200 records

=== CHECK CALCULATIONS ===
Total tasks: 500
Total allocated cost: $9,400.00

Successful tasks: 420
Failed tasks: 80

=== FIRST 10 RECORDS ===

| date       | start               | is_weekend | task_id | service_name | node_type | success | task_count | total_seconds | duration_ratio | daily_node_cost | allocated_cost | cost_per_generation | cost_per_second |
|------------|---------------------|------------|---------|--------------|-----------|---------|------------|---------------|----------------|-----------------|----------------|---------------------|-----------------|
| 2025-06-01 | 2025-06-01 08:00:00 | False      | 201     | gen_image    | a100      | True    | 1          | 30            | 0.06           | 220.00          | 13.20          | 13.20               | NaN             |
| 2025-06-01 | 2025-06-01 08:05:00 | False      | 202     | gen_text     | l4        | True    | 1          | 45            | 0.08           | 36.00           | 2.88           | 2.88                | NaN             |
| 2025-06-01 | 2025-06-01 08:10:00 | False      | 203     | upscale      | a100      | False   | 1          | 15            | 0.03           | 220.00          | 6.60           | NaN                 | 0.44            |
| 2025-06-01 | 2025-06-01 08:20:00 | False      | 204     | gen_image    | a100      | True    | 1          | 70            | 0.14           | 220.00          | 30.80          | 30.80               | NaN             |
| 2025-06-01 | 2025-06-01 08:30:00 | False      | 205     | gen_video    | l4        | True    | 1          | 60            | 0.11           | 36.00           | 3.96           | 3.96                | NaN             |
| ...        | ...                 | ...        | ...     | ...          | ...       | ...     | ...        | ...           | ...            | ...             | ...            | ...                 | ...             |

## Step 10: Current data analysis

### 10.1: Comprehensive coverage analysis of node costs

We analyze how fully the total node costs are allocated to tasks and services:
1. Totals
2. Dynamics over time
3. By node type
4. Weekdays vs weekends
5. By services

In [ ]:
print("Comprehensive analysis of node cost coverage")
print("=" * 60)

# 1. Totals
total_node_cost = df_daily_costs['daily_cost'].sum()
total_allocated_cost = df_cost_per_gen_daily['allocated_cost'].sum()
coverage_percentage = (total_allocated_cost / total_node_cost) * 100

print("TOTALS:")
print(f"  Node costs for entire period: ${total_node_cost:,.2f}")
print(f"  Allocated costs to tasks: ${total_allocated_cost:,.2f}")
print(f"  Coverage percentage: {coverage_percentage:.2f}%")
print(f"  Difference (nodes - tasks): ${total_node_cost - total_allocated_cost:,.2f}")

# 2. Daily coverage dynamics
print("\nDAILY COVERAGE DYNAMICS:")
daily_coverage = df_daily_costs.groupby('date').agg({
    'daily_cost': 'sum'
}).reset_index()

daily_allocated = df_cost_per_gen_daily.groupby(['date', 'is_weekend']).agg({
    'allocated_cost': 'sum'
}).reset_index()

daily_coverage_merged = daily_coverage.merge(daily_allocated, on='date', how='left').fillna(0)
daily_coverage_merged['coverage_pct'] = (daily_coverage_merged['allocated_cost'] / daily_coverage_merged['daily_cost']) * 100
daily_coverage_merged['difference'] = daily_coverage_merged['daily_cost'] - daily_coverage_merged['allocated_cost']

print(f"  Avg. daily coverage: {daily_coverage_merged['coverage_pct'].mean():.2f}%")
print(f"  Min daily coverage: {daily_coverage_merged['coverage_pct'].min():.2f}%")
print(f"  Max daily coverage: {daily_coverage_merged['coverage_pct'].max():.2f}%")
print(f"  Days with full coverage: {(daily_coverage_merged['coverage_pct'] >= 99).sum()}")
print(f"  Days with low coverage (<50%): {(daily_coverage_merged['coverage_pct'] < 50).sum()}")

# 3. By node type
print("\nBY NODE TYPE:")
node_type_analysis = df_daily_costs.groupby('node_type').agg({'daily_cost': 'sum'}).reset_index()
node_type_allocated = df_cost_per_gen_daily.groupby('node_type').agg({'allocated_cost': 'sum'}).reset_index()
node_type_merged = node_type_analysis.merge(node_type_allocated, on='node_type', how='left').fillna(0)
node_type_merged['coverage_pct'] = (node_type_merged['allocated_cost'] / node_type_merged['daily_cost']) * 100
node_type_merged['difference'] = node_type_merged['daily_cost'] - node_type_merged['allocated_cost']

for _, row in node_type_merged.iterrows():
    print(f"  {row['node_type'].upper()}:")
    print(f"    Node costs: ${row['daily_cost']:,.2f}")
    print(f"    Allocated: ${row['allocated_cost']:,.2f}")
    print(f"    Coverage: {row['coverage_pct']:.2f}%")
    print(f"    Difference: ${row['difference']:,.2f}")

# 4. By weekday/weekend
print("\nBY WEEKDAY / WEEKEND:")
weekday_analysis = df_daily_costs.groupby('is_weekend').agg({'daily_cost': 'sum'}).reset_index()
weekday_allocated = df_cost_per_gen_daily.groupby('is_weekend').agg({'allocated_cost': 'sum'}).reset_index()
weekday_merged = weekday_analysis.merge(weekday_allocated, on='is_weekend', how='left').fillna(0)
weekday_merged['coverage_pct'] = (weekday_merged['allocated_cost'] / weekday_merged['daily_cost']) * 100
weekday_merged['difference'] = weekday_merged['daily_cost'] - weekday_merged['allocated_cost']

for _, row in weekday_merged.iterrows():
    day_type = "Weekend" if row['is_weekend'] else "Weekday"
    print(f"  {day_type}:")
    print(f"    Node costs: ${row['daily_cost']:,.2f}")
    print(f"    Allocated: ${row['allocated_cost']:,.2f}")
    print(f"    Coverage: {row['coverage_pct']:.2f}%")
    print(f"    Difference: ${row['difference']:,.2f}")

# 5. By ML services
print("\nBY ML SERVICES:")
service_analysis = df_cost_per_gen_daily.groupby('service_name').agg({
    'allocated_cost': 'sum',
    'task_count': 'sum'
}).reset_index()

service_analysis = service_analysis.sort_values('allocated_cost', ascending=False)
service_analysis['cost_per_task'] = service_analysis['allocated_cost'] / service_analysis['task_count']

print("  Top-5 services by cost:")
for i, (_, row) in enumerate(service_analysis.head().iterrows(), 1):
    print(f"    {i}. {row['service_name']}:")
    print(f"       Total cost: ${row['allocated_cost']:,.2f}")
    print(f"       Tasks: {row['task_count']:,}")
    print(f"       Cost per task: ${row['cost_per_task']:.6f}")

# 6. Utilization efficiency
print("\nUTILIZATION EFFICIENCY:")
total_node_hours = df_daily_costs['node_hours'].sum()
total_task_seconds = df_intervals['duration_seconds'].sum()
total_task_hours = total_task_seconds / 3600
utilization_rate = (total_task_hours / total_node_hours) * 100

print(f"  Total node runtime: {total_node_hours:,.0f} node-hours")
print(f"  Total task execution time: {total_task_hours:,.0f} hours")
print(f"  Utilization rate: {utilization_rate:.2f}%")

# 7. Coverage details per day
print("\nCOVERAGE TABLE (first 10 days):")
display_cols = ['date', 'daily_cost', 'allocated_cost', 'coverage_pct', 'difference']
display(daily_coverage_merged[display_cols].head(10))

# 8. Coverage dynamics chart
print("\nCOVERAGE DYNAMICS CHART:")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Chart 1: Node costs vs allocated costs
ax1.plot(daily_coverage_merged['date'], daily_coverage_merged['daily_cost'],
         label='Node costs', marker='o', linewidth=2, color='red')
ax1.plot(daily_coverage_merged['date'], daily_coverage_merged['allocated_cost'],
         label='Allocated costs', marker='s', linewidth=2, color='blue')
ax1.fill_between(daily_coverage_merged['date'],
                 daily_coverage_merged['daily_cost'],
                 daily_coverage_merged['allocated_cost'],
                 alpha=0.3, color='orange', label='Difference')
ax1.set_title('Dynamics: Node costs vs Allocated costs')
ax1.set_ylabel('Cost ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Chart 2: Coverage percentage
ax2.plot(daily_coverage_merged['date'], daily_coverage_merged['coverage_pct'],
         marker='o', linewidth=2, color='green')
ax2.axhline(y=100, color='red', linestyle='--', alpha=0.7, label='100% coverage')
ax2.axhline(y=coverage_percentage, color='orange', linestyle='--', alpha=0.7,
            label=f'Avg. coverage: {coverage_percentage:.1f}%')
ax2.set_title('Coverage percentage dynamics')
ax2.set_ylabel('Coverage (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nCOVERAGE ANALYSIS COMPLETED!")
print("Key findings:")
print(f"   - Total coverage: {coverage_percentage:.1f}%")
print(f"   - Average daily coverage: {daily_coverage_merged['coverage_pct'].mean():.1f}%")
print(f"   - Max daily difference: ${daily_coverage_merged['difference'].max():,.2f}")
print(f"   - Node utilization rate: {utilization_rate:.1f}%")

**Mock output (illustrative example):**

TOTALS:
- Node costs for entire period: `$9,400.00`
- Allocated costs to tasks: `$9,200.00`
- Coverage percentage: `97.87%`
- Difference: `$200.00`

DAILY COVERAGE DYNAMICS:
- Avg. daily coverage: `97.5%`
- Min daily coverage: `80.0%`
- Max daily coverage: `100.0%`
- Days with full coverage: `45`
- Days with low coverage (<50%): `0`

BY NODE TYPE:
- A100: Node costs `$6,600.00`, Allocated `$6,500.00`, Coverage `98.5%`, Difference `$100.00`
- L4:   Node costs `$2,800.00`, Allocated `$2,700.00`, Coverage `96.4%`, Difference `$100.00`

BY WEEKDAY / WEEKEND:
- Weekday: Node costs `$7,500.00`, Allocated `$7,400.00`, Coverage `98.7%`, Difference `$100.00`
- Weekend: Node costs `$1,900.00`, Allocated `$1,800.00`, Coverage `94.7%`, Difference `$100.00`

BY ML SERVICES (top-3):
1. gen_image → `$3,500.00` / `1,200` tasks → `$2.92` per task
2. gen_text  → `$2,200.00` / `900` tasks → `$2.44` per task
3. upscale   → `$1,500.00` / `400` tasks → `$3.75` per task

UTILIZATION EFFICIENCY:
- Total node runtime: `1,200 node-hours`
- Total task execution time: `597 hours`
- Utilization rate: `49.8%`

COVERAGE TABLE (first 5 days):

| `date`     | `daily_cost` | `allocated_cost` | `coverage_pct` | `difference` |
|------------|--------------|------------------|----------------|--------------|
| 2025-06-01 | `256.00`     | `250.00`         | `97.7%`        | `6.00`       |
| 2025-06-02 | `256.00`     | `255.00`         | `99.6%`        | `1.00`       |
| 2025-06-03 | `256.00`     | `250.00`         | `97.7%`        | `6.00`       |
| 2025-06-04 | `256.00`     | `248.00`         | `96.9%`        | `8.00`       |
| 2025-06-05 | `256.00`     | `252.00`         | `98.4%`        | `4.00`       |

COVERAGE DYNAMICS CHART:
- Chart 1: Node costs vs allocated costs over time
- Chart 2: Coverage percentage over time

COVERAGE ANALYSIS COMPLETED!
Key findings:
- Total coverage: `97.9%`
- Avg. daily coverage: `97.5%`
- Max daily difference: `$12.00`
- Node utilization rate: `79.2%`

### 10.2: Detailed node utilization analysis

We calculate how efficiently node resources are consumed.
**Idle time** is defined as the sum of periods when a node was running but no tasks were executed.

Important: any active node is billed regardless of actual workload.

In [ ]:
print("DETAILED ANALYSIS OF NODE UTILIZATION")
print("=" * 70)

# 1. Hourly analysis across the entire period
print("HOURLY ANALYSIS ACROSS THE ENTIRE PERIOD:")

df_cost_per_gen_daily['hour'] = df_cost_per_gen_daily['start'].dt.hour

# Group by hour and day type, aggregate usage
hourly_usage = df_cost_per_gen_daily.groupby(['hour', 'is_weekend']).agg({
    'total_seconds': 'sum',
    'allocated_cost': 'sum'
}).reset_index()

# Compute node-hours for the whole period for each hour
def get_node_hours_by_hour_for_period():
    exact_hours = []
    start_date = pd.to_datetime('2025-06-01')

    if df_daily_tasks['date'].dtype == 'object':
        max_date = pd.to_datetime(df_daily_tasks['date'].max())
    else:
        max_date = df_daily_tasks['date'].max()

    end_date = pd.to_datetime(max_date) if not isinstance(max_date, pd.Timestamp) else max_date
    total_days = (end_date - start_date).days + 1

    print(f"Analysis period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"Total days: {total_days}")

    for is_weekend in [False, True]:
        for node_type in ['a100', 'l4']:
            subset = df_continuous[
                (df_continuous['is_weekend'] == is_weekend) &
                (df_continuous['node_type'] == node_type)
            ]

            for _, row in subset.iterrows():
                hour = row['hour']
                nodes_count = row['nodes_count']
                days_count = int(total_days * (2/7 if is_weekend else 5/7))

                exact_hours.append({
                    'hour': hour,
                    'is_weekend': is_weekend,
                    'node_type': node_type,
                    'nodes_count': nodes_count,
                    'days_count': days_count,
                    'node_hours_total': nodes_count * days_count
                })

    return pd.DataFrame(exact_hours)

# Node-hours for the entire period
node_schedule_period = get_node_hours_by_hour_for_period()
hourly_schedule_period = node_schedule_period.groupby(['hour', 'is_weekend'])['node_hours_total'].sum().reset_index()
hourly_schedule_period.columns = ['hour', 'is_weekend', 'total_node_hours_period']

# Merge usage with schedule
hourly_usage = hourly_usage.merge(hourly_schedule_period, on=['hour', 'is_weekend'], how='left').fillna(0)

# Compute utilization metrics
hourly_usage['hours_used_period'] = hourly_usage['total_seconds'] / 3600
hourly_usage['utilization_pct'] = (
    hourly_usage['hours_used_period'] / hourly_usage['total_node_hours_period']
) * 100 if hourly_usage['total_node_hours_period'].sum() > 0 else 0

hourly_usage_weekdays = hourly_usage[hourly_usage['is_weekend'] == False]
hourly_usage_weekends = hourly_usage[hourly_usage['is_weekend'] == True]

print("\nSUMMARY FOR THE ENTIRE PERIOD:")
print(f"  Total task runtime: {hourly_usage['hours_used_period'].sum():.1f} hours")
print(f"  Total node runtime: {hourly_usage['total_node_hours_period'].sum():.1f} node-hours")
print(f"  Overall utilization: {(hourly_usage['hours_used_period'].sum() / hourly_usage['total_node_hours_period'].sum()) * 100:.1f}%")

print("\nWEEKDAYS - Top-5 busiest hours:")
for _, row in hourly_usage_weekdays.nlargest(5, 'hours_used_period').iterrows():
    print(f"    {row['hour']:02d}:00 - {row['hours_used_period']:.1f}h, {row['utilization_pct']:.1f}% (node-hours: {row['total_node_hours_period']:.1f})")

print("\nWEEKDAYS - Top-5 idle hours:")
for _, row in hourly_usage_weekdays.nsmallest(5, 'hours_used_period').iterrows():
    print(f"    {row['hour']:02d}:00 - {row['hours_used_period']:.1f}h, {row['utilization_pct']:.1f}% (node-hours: {row['total_node_hours_period']:.1f})")

print("\nWEEKENDS - Top-5 busiest hours:")
for _, row in hourly_usage_weekends.nlargest(5, 'hours_used_period').iterrows():
    print(f"    {row['hour']:02d}:00 - {row['hours_used_period']:.1f}h, {row['utilization_pct']:.1f}% (node-hours: {row['total_node_hours_period']:.1f})")

print("\nWEEKENDS - Top-5 idle hours:")
for _, row in hourly_usage_weekends.nsmallest(5, 'hours_used_period').iterrows():
    print(f"    {row['hour']:02d}:00 - {row['hours_used_period']:.1f}h, {row['utilization_pct']:.1f}% (node-hours: {row['total_node_hours_period']:.1f})")

**Mock output (illustrative example):**

Analysis period: 2025-06-01 to 2025-08-31
Total days: `92`

SUMMARY FOR THE ENTIRE PERIOD:
- Total task runtime: `950.0 hours`
- Total node runtime: `1,200.0 node-hours`
- Overall utilization: `79.2%`

WEEKDAYS - Top-5 busiest hours:
- 14:00 → `85.0h`, `95.0%` utilization (`90.0 node-hours`)
- 15:00 → `80.0h`, `90.0%` utilization (`89.0 node-hours`)
- 13:00 → `75.0h`, `88.2%` utilization (`85.0 node-hours`)
- 16:00 → `70.0h`, `84.0%` utilization (`83.0 node-hours`)
- 12:00 → `65.0h`, `82.0%` utilization (`79.0 node-hours`)

WEEKDAYS - Top-5 idle hours:
- 03:00 → `5.0h`, `6.0%` utilization (`80.0 node-hours`)
- 04:00 → `6.0h`, `7.5%` utilization (`80.0 node-hours`)
- 02:00 → `8.0h`, `10.0%` utilization (`80.0 node-hours`)
- 01:00 → `10.0h`, `12.5%` utilization (`80.0 node-hours`)
- 05:00 → `12.0h`, `15.0%` utilization (`80.0 node-hours`)

WEEKENDS - Top-5 busiest hours:
- 20:00 → `40.0h`, `88.0%` utilization (`45.0 node-hours`)
- 21:00 → `35.0h`, `85.0%` utilization (`41.0 node-hours`)
- 19:00 → `32.0h`, `80.0%` utilization (`40.0 node-hours`)
- 22:00 → `30.0h`, `78.0%` utilization (`38.0 node-hours`)
- 18:00 → `28.0h`, `75.0%` utilization (`37.0 node-hours`)

WEEKENDS - Top-5 idle hours:
- 04:00 → `2.0h`, `5.0%` utilization (`40.0 node-hours`)
- 03:00 → `3.0h`, `7.0%` utilization (`42.0 node-hours`)
- 05:00 → `4.0h`, `9.5%` utilization (`42.0 node-hours`)
- 02:00 → `5.0h`, `12.0%` utilization (`42.0 node-hours`)
- 06:00 → `6.0h`, `14.0%` utilization (`43.0 node-hours`)

In [ ]:
# 2. Analysis by day of the week
print("\nANALYSIS BY DAY OF THE WEEK:")

df_cost_per_gen_daily['weekday_name'] = df_cost_per_gen_daily['start'].dt.day_name()

# Group by weekday and aggregate usage
weekday_hours = df_cost_per_gen_daily.groupby('weekday_name').agg({
    'total_seconds': 'sum',
    'allocated_cost': 'sum'
}).reset_index()

weekday_hours['hours_used'] = weekday_hours['total_seconds'] / 3600

# Map weekday names to numbers (0=Monday, 6=Sunday)
weekday_mapping = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3,
    'Friday': 4, 'Saturday': 5, 'Sunday': 6
}

weekday_hours['weekday_num'] = weekday_hours['weekday_name'].map(weekday_mapping)

# Get node-hours per weekday across the entire period
weekday_node_hours_df = df_daily_costs.copy()
weekday_node_hours_df['weekday_name'] = pd.to_datetime(weekday_node_hours_df['date']).dt.day_name()

weekday_node_hours_df = weekday_node_hours_df.groupby(['weekday_name', 'is_weekend']).agg({
    'node_hours': 'sum'
}).reset_index()

weekday_node_hours_df['weekday_num'] = weekday_node_hours_df['weekday_name'].map(weekday_mapping)

# Merge with usage data
weekday_hours = weekday_hours.merge(weekday_node_hours_df, on='weekday_num', how='left')

# Calculate utilization percentage per weekday
weekday_hours['utilization_pct'] = (
    weekday_hours['hours_used'] / weekday_hours['node_hours']
) * 100 if weekday_hours['node_hours'].sum() > 0 else 0

# Sort by weekday number for correct order
weekday_hours = weekday_hours.sort_values('weekday_num')

print("\nUTILIZATION ANALYSIS BY WEEKDAY (FULL PERIOD):")
for _, row in weekday_hours.iterrows():
    day_type = "Weekend" if row['is_weekend'] else "Weekday"
    print(f"  {row['weekday_name']} ({day_type}):")
    print(f"    Task runtime: {row['hours_used']:.1f}h")
    print(f"    Node-hours: {row['node_hours']:.1f}")
    print(f"    Utilization: {row['utilization_pct']:.1f}%\n")

# Show summary table
print("\nSUMMARY TABLE BY WEEKDAY:")
display_cols = ['weekday_name', 'is_weekend', 'hours_used', 'node_hours', 'utilization_pct']
display(weekday_hours[display_cols].round(2))

**Mock output (illustrative example):**

UTILIZATION ANALYSIS BY WEEKDAY (FULL PERIOD):

  Monday (Weekday):
    Task runtime: 120.0h
    Node-hours: 200.0
    Utilization: 60.0%

  Tuesday (Weekday):
    Task runtime: 125.0h
    Node-hours: 200.0
    Utilization: 62.5%

  Wednesday (Weekday):
    Task runtime: 115.0h
    Node-hours: 200.0
    Utilization: 57.5%

  Thursday (Weekday):
    Task runtime: 130.0h
    Node-hours: 200.0
    Utilization: 65.0%

  Friday (Weekday):
    Task runtime: 110.0h
    Node-hours: 200.0
    Utilization: 55.0%

  Saturday (Weekend):
    Task runtime: 70.0h
    Node-hours: 180.0
    Utilization: 38.9%

  Sunday (Weekend):
    Task runtime: 75.0h
    Node-hours: 180.0
    Utilization: 41.7%


SUMMARY TABLE BY WEEKDAY:

| `weekday_name` | `is_weekend` | `hours_used` | `node_hours` | `utilization_pct` |
|----------------|--------------|--------------|--------------|-------------------|
| Monday         | False        | 120.0        | 200.0        | 60.0              |
| Tuesday        | False        | 125.0        | 200.0        | 62.5              |
| Wednesday      | False        | 115.0        | 200.0        | 57.5              |
| Thursday       | False        | 130.0        | 200.0        | 65.0              |
| Friday         | False        | 110.0        | 200.0        | 55.0              |
| Saturday       | True         | 70.0         | 180.0        | 38.9              |
| Sunday         | True         | 75.0         | 180.0        | 41.7              |

In [ ]:
# 3. Idle time analysis
print("\nIDLE TIME ANALYSIS:")
total_node_hours = df_daily_costs['node_hours'].sum()
total_task_hours = df_daily_tasks['total_seconds'].sum() / 3600
idle_hours = total_node_hours - total_task_hours

print(f"  Total node runtime: {total_node_hours:,.0f} node-hours")
print(f"  Task runtime: {total_task_hours:,.0f} hours")
print(f"  Idle time: {idle_hours:,.0f} hours")
print(f"  Idle percentage: {(idle_hours / total_node_hours) * 100:.1f}%")

# 4. Analysis by node type
print("\nANALYSIS BY NODE TYPE:")
node_type_efficiency = df_daily_costs.groupby('node_type').agg({
    'node_hours': 'sum',
    'daily_cost': 'sum'
}).reset_index()

node_type_tasks = df_cost_per_gen_daily.groupby('node_type').agg({
    'allocated_cost': 'sum'
}).reset_index()

node_type_efficiency = node_type_efficiency.merge(node_type_tasks, on='node_type', how='left').fillna(0)
node_type_efficiency['cost_per_hour'] = node_type_efficiency['daily_cost'] / node_type_efficiency['node_hours']
node_type_efficiency['efficiency_ratio'] = node_type_efficiency['allocated_cost'] / node_type_efficiency['daily_cost']

for _, row in node_type_efficiency.iterrows():
    print(f"  {row['node_type'].upper()}:")
    print(f"    Cost per hour: ${row['cost_per_hour']:.4f}")
    print(f"    Efficiency: {row['efficiency_ratio']*100:.1f}%")

# 5. Gap analysis
print("\nTASK GAP ANALYSIS:")
df_intervals_sorted = df_intervals.sort_values(['date', 'start'])
df_intervals_sorted['next_start'] = df_intervals_sorted.groupby('date')['start'].shift(-1)
df_intervals_sorted['gap_hours'] = (df_intervals_sorted['next_start'] - df_intervals_sorted['end']).dt.total_seconds() / 3600

gaps = df_intervals_sorted[df_intervals_sorted['gap_hours'] > 0]
if not gaps.empty:
    print(f"  Avg. gap between tasks: {gaps['gap_hours'].mean():.2f} hours")
    print(f"  Max gap: {gaps['gap_hours'].max():.2f} hours")
    print(f"  Total number of gaps: {len(gaps)}")
else:
    print("  Tasks run continuously")

# 6. Optimization recommendations
print("\nOPTIMIZATION RECOMMENDATIONS:")

night_hours = hourly_usage[hourly_usage['hour'].isin([0,1,2,3,4,5,6])]
night_utilization = night_hours['utilization_pct'].mean()
print(f"  Nighttime utilization (0-6h): {night_utilization:.1f}%")
if night_utilization < 10:
    print("    Low nighttime utilization - consider shutting down nodes")
elif night_utilization < 30:
    print("    Moderate nighttime utilization - consider partial shutdown")

weekend_utilization = weekday_hours[weekday_hours['weekday_name'].isin(['Saturday', 'Sunday'])]['utilization_pct'].mean()
print(f"  Weekend utilization: {weekend_utilization:.1f}%")
if weekend_utilization < 20:
    print("    Low weekend utilization - consider shutting down nodes")

# 7. Visualization
print("\nVISUALIZATION OF HOURLY AND WEEKDAY UTILIZATION:")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

ax1.bar(hourly_usage['hour'], hourly_usage['utilization_pct'], color='skyblue', alpha=0.7, edgecolor='navy')
ax1.set_title('Node utilization by hour of day')
ax1.set_xlabel('Hour of day')
ax1.set_ylabel('Utilization (%)')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(0, 24))

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_usage_ordered = weekday_hours.set_index('weekday_name').reindex(weekday_order)

ax2.bar(range(len(weekday_usage_ordered)), weekday_usage_ordered['utilization_pct'],
        color='lightcoral', alpha=0.7, edgecolor='darkred')
ax2.set_title('Node utilization by weekday')
ax2.set_xlabel('Weekday')
ax2.set_ylabel('Utilization (%)')
ax2.set_xticks(range(len(weekday_usage_ordered)))
ax2.set_xticklabels(weekday_usage_ordered.index, rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 8. Detailed hourly stats
hourly_detailed = hourly_usage.copy()
hourly_detailed['cost_per_hour'] = hourly_detailed['allocated_cost'] / hourly_detailed['hours_used_period']
hourly_detailed['efficiency_score'] = hourly_detailed['utilization_pct'] / hourly_detailed['utilization_pct'].max()

print("\nUTILIZATION ANALYSIS COMPLETED!")
print("Key insights:")
print(f"   - Busiest hour: {hourly_usage.loc[hourly_usage['utilization_pct'].idxmax(), 'hour']:02d}:00")
print(f"   - Idlest hour: {hourly_usage.loc[hourly_usage['utilization_pct'].idxmin(), 'hour']:02d}:00")
print(f"   - Busiest day: {weekday_hours.loc[weekday_hours['utilization_pct'].idxmax(), 'weekday_name']}")
print(f"   - Potential savings: {idle_hours:.0f} idle hours")

**Mock output (illustrative example):**

IDLE TIME ANALYSIS:
  Total node runtime: 12,000 node-hours
  Task runtime: 6,000 hours
  Idle time: 6,000 hours
  Idle percentage: 50.0%

ANALYSIS BY NODE TYPE:
  A100:
    Cost per hour: $11.0000
    Efficiency: 52.0%
  L4:
    Cost per hour: $1.0000
    Efficiency: 48.0%

TASK GAP ANALYSIS:
  Avg. gap between tasks: 0.80 hours
  Max gap: 5.20 hours
  Total number of gaps: 430

OPTIMIZATION RECOMMENDATIONS:
  Nighttime utilization (0-6h): 8.5%
    Low nighttime utilization - consider shutting down nodes
  Weekend utilization: 40.0%

VISUALIZATION OF HOURLY AND WEEKDAY UTILIZATION:
(Hourly bar chart + weekday bar chart)

Key insights:
   - Busiest hour: 14:00
   - Idlest hour: 04:00
   - Busiest day: Wednesday
   - Potential savings: 6,000 idle hours

## Step 11: Final Tables and Statistics

We generate final summary tables with analysis results:
1. **Summary by ML services** – generation costs per service
2. **Summary by node types** – overall statistics for A100 and L4
3. **Split statistics** – successful vs failed tasks
4. **Top services** by different metrics

**Key metrics**:
- Successful tasks: cost per generation
- Failed tasks: cost per second

In [ ]:
# Final summary tables
print("FINAL ANALYSIS RESULTS")
print("=" * 50)

# Helper: calculate distribution stats
def calc_stats(series, weights=None):
    if series.empty:
        return {}
    return {
        "count": len(series),
        "mean": series.mean(),
        "median": series.median(),
        "p05": series.quantile(0.05),
        "p25": series.quantile(0.25),
        "p75": series.quantile(0.75),
        "p90": series.quantile(0.90),
        "p95": series.quantile(0.95),
        "weighted_mean": (series * weights).sum() / weights.sum() if weights is not None else None
    }

# Table 0: Summary by all ML services
print("\n=== TABLE 0: SUMMARY BY ALL ML SERVICES ===")
all_services = df_cost_per_gen_daily

summary_by_service = all_services.groupby('service_name').agg({
    'task_count': 'sum',
    'allocated_cost': 'sum',
    'cost_per_generation': ['mean', 'std']
}).round(6)

summary_by_service.columns = ['total_tasks', 'total_allocated_cost', 'avg_cost_per_gen', 'std_cost_per_gen']

summary_by_service['weighted_avg_cost'] = (
    summary_by_service['total_allocated_cost'] / summary_by_service['total_tasks']
).round(6)

summary_by_service = summary_by_service.sort_values('total_tasks', ascending=False)
display(summary_by_service.head(15))

# Table 1: Successful tasks only
print("\n=== TABLE 1: SUMMARY BY ML SERVICES (SUCCESSFUL) ===")
success_services = df_cost_per_gen_daily[df_cost_per_gen_daily['success'] & df_cost_per_gen_daily['cost_per_generation'].notna()]

summary_by_service_success = success_services.groupby('service_name').agg({
    'task_count': 'sum',
    'allocated_cost': 'sum',
    'cost_per_generation': ['mean', 'std']
}).round(6)

summary_by_service_success.columns = ['total_tasks', 'total_allocated_cost', 'avg_cost_per_gen', 'std_cost_per_gen']

summary_by_service_success['weighted_avg_cost'] = (
    summary_by_service_success['total_allocated_cost'] / summary_by_service_success['total_tasks']
).round(6)

summary_by_service_success = summary_by_service_success.sort_values('total_tasks', ascending=False)
display(summary_by_service_success.head(15))

# Table 2: Failed tasks only
print("\n=== TABLE 2: SUMMARY BY ML SERVICES (FAILED) ===")
fail_services = df_cost_per_gen_daily[~df_cost_per_gen_daily['success'] & df_cost_per_gen_daily['cost_per_second'].notna()]

summary_by_service_fail = fail_services.groupby('service_name').agg({
    'task_count': 'sum',
    'total_duration_seconds': 'sum',
    'allocated_cost': 'sum',
    'cost_per_second': ['mean', 'std']
}).round(6)

summary_by_service_fail.columns = ['total_tasks', 'total_seconds', 'total_allocated_cost', 'avg_cost_per_second', 'std_cost_per_second']

summary_by_service_fail['weighted_avg_cost_per_second'] = (
    summary_by_service_fail['total_allocated_cost'] / summary_by_service_fail['total_seconds']
).round(6)

summary_by_service_fail = summary_by_service_fail.sort_values('total_tasks', ascending=False)
display(summary_by_service_fail.head(15))

# Table 3: Summary by node type
print("\n=== TABLE 3: SUMMARY BY NODE TYPE ===")
summary_by_node_type = df_cost_per_gen_daily.groupby('node_type').agg({
    'task_count': 'sum',
    'allocated_cost': 'sum'
}).round(6)

summary_by_node_type['total_cost'] = summary_by_node_type['allocated_cost']
display(summary_by_node_type)

# Global stats
print("\n=== GLOBAL STATS ===")
total_tasks_all = df_cost_per_gen_daily['task_count'].sum()
total_cost_all = df_cost_per_gen_daily['allocated_cost'].sum()

print(f"Total tasks: {total_tasks_all:,}")
print(f"Total node costs: ${total_cost_all:.2f}")
print(f"Analysis period: from 2025-06-01 to {datetime.now().strftime('%Y-%m-%d')}")

# Cost distribution stats
success_costs = df_cost_per_gen_daily.loc[df_cost_per_gen_daily.success & df_cost_per_gen_daily.cost_per_generation.notna(), "cost_per_generation"]
fail_costs = df_cost_per_gen_daily.loc[~df_cost_per_gen_daily.success & df_cost_per_gen_daily.cost_per_second.notna(), "cost_per_second"]

print("\n=== COST DISTRIBUTION STATS ===")
print("Successful tasks:")
print(calc_stats(success_costs))

print("\nFailed tasks (cost per second):")
print(calc_stats(fail_costs))

**Mock output (illustrative example):**

=== TABLE 0: SUMMARY BY ALL ML SERVICES ===
| service_name    | total_tasks | total_allocated_cost | avg_cost_per_gen | std_cost_per_gen | weighted_avg_cost |
|-----------------|-------------|----------------------|------------------|------------------|-------------------|
| img_gen_v2      | 45,000      | 250,000.00           | 0.0060           | 0.0012           | 0.0056            |
| upscale_ai      | 12,500      | 80,000.00            | 0.0072           | 0.0021           | 0.0064            |
| text2img_beta   | 8,200       | 42,000.00            | 0.0054           | 0.0018           | 0.0051            |

=== TABLE 1: SUMMARY BY ML SERVICES (SUCCESSFUL) ===
(similar structure, only successful tasks included)

=== TABLE 2: SUMMARY BY ML SERVICES (FAILED) ===
| service_name    | total_tasks | total_seconds | total_allocated_cost | avg_cost_per_second | std_cost_per_second | weighted_avg_cost_per_second |
|-----------------|-------------|---------------|----------------------|---------------------|---------------------|-------------------------------|
| img_gen_v2      | 4,000       | 180,000       | 20,000.00            | 0.1200              | 0.0450              | 0.1111                        |
| upscale_ai      | 1,200       | 40,000        | 3,800.00             | 0.0950              | 0.0310              | 0.0945                        |

=== TABLE 3: SUMMARY BY NODE TYPE ===
| node_type | task_count | allocated_cost | total_cost |
|-----------|------------|----------------|------------|
| a100      | 38,000     | 220,000.00     | 220,000.00 |
| l4        | 31,000     | 152,000.00     | 152,000.00 |

=== GLOBAL STATS ===
- Total tasks: 69,000
- Total node costs: $372,000.00
- Analysis period: from 2025-06-01 to 2025-10-23

=== COST DISTRIBUTION STATS ===
Successful tasks:
`{'count': 65,000, 'mean': 0.0061, 'median': 0.0058, 'p05': 0.0030, 'p25': 0.0045, 'p75': 0.0074, 'p90': 0.0091, 'p95': 0.0100, 'weighted_mean': 0.0059}`

Failed tasks (cost per second):
`{'count': 4,000, 'mean': 0.112, 'median': 0.105, 'p05': 0.050, 'p25': 0.080, 'p75': 0.135, 'p90': 0.150, 'p95': 0.170, 'weighted_mean': 0.111}`

## Step 11: Results Visualization

We create visualizations to present the analysis results more clearly:
1. **Daily dynamics of successful generation costs**
2. **Daily dynamics of failed task costs**
3. **Cost comparison by node type**
4. **Top services by total cost**

In [ ]:
# Create visualizations
print("Creating plots...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Dynamics of average successful generation cost
success_by_date = df_cost_per_gen_daily[df_cost_per_gen_daily.success].groupby("date")["cost_per_generation"].mean()
success_by_date.plot(kind='line', marker="o", ax=axes[0,0], color='green', linewidth=2)
axes[0,0].set_title('Dynamics of Avg. Successful Generation Cost\n(from 2025-06-01)')
axes[0,0].set_ylabel('Cost per generation ($)')
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Dynamics of average failed task cost (per second)
fail_by_date = df_cost_per_gen_daily[~df_cost_per_gen_daily.success].groupby("date")["cost_per_second"].mean()
fail_by_date.plot(kind='line', marker="o", ax=axes[0,1], color="red", linewidth=2)
axes[0,1].set_title('Dynamics of Avg. Failed Task Cost\n(cost per second)')
axes[0,1].set_ylabel('Cost per second ($)')
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Total cost by node type
node_type_costs = df_cost_per_gen_daily.groupby('node_type')['allocated_cost'].sum()
node_type_costs.plot(kind='bar', ax=axes[1,0], color=['#ff7f0e', '#1f77b4'])
axes[1,0].set_title('Total Cost by Node Type')
axes[1,0].set_ylabel('Total cost ($)')
axes[1,0].tick_params(axis='x', rotation=0)
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Top-10 services by weighted avg cost (successful generations)
top_10_success = summary_by_service_success.head(10)
top_10_success['weighted_avg_cost'].plot(kind='bar', ax=axes[1,1], color='lightgreen')
axes[1,1].set_title('Top-10 Services by Successful Generation Cost')
axes[1,1].set_ylabel('Cost per generation ($)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nAnalysis completed!")
print("All summary tables and plots created")
print("Node costs allocated proportionally to runtime")
print("Separate pricing applied: per-task for successful, per-second for failed")
print("Weighted averages shown for each ML service")

**Mock visualizations:**

1. **Dynamics of Avg. Successful Generation Cost**
   - Smooth line around $0.006–0.007 per generation
   - Slight downward trend as autoscaling reduces idle costs

2. **Dynamics of Avg. Failed Task Cost (per second)**
   - Fluctuates more, around $0.09–0.13 per second
   - Occasional spikes when failures cluster

3. **Total Cost by Node Type**
   - A100: ~$220,000
   - L4: ~$152,000

4. **Top-10 Services by Successful Generation Cost**
   - `img_gen_v2`: ~$0.0056 per generation
   - `upscale_ai`: ~$0.0064 per generation
   - `text2img_beta`: ~$0.0051 per generation
   - (others descending…)

## Saving Results

Save the main analysis outputs into CSV files for further use.

In [ ]:
# Save results
try:
    # All tasks
    summary_by_service.to_csv('../data/gen_price_by_service_enhanced.csv')
    print("Saved: gen_price_by_service_enhanced.csv")

    # Successful tasks
    summary_by_service_success.to_csv('../data/gen_price_by_service_success_enhanced.csv')
    print("Saved: gen_price_by_service_success_enhanced.csv")

    # Failed tasks
    summary_by_service_fail.to_csv('../data/gen_price_by_service_fail_enhanced.csv')
    print("Saved: gen_price_by_service_fail_enhanced.csv")

    # Detailed daily costs per generation
    df_cost_per_gen_daily.to_csv('../data/gen_cost_detailed_enhanced.csv', index=False)
    print("Saved: gen_cost_detailed_enhanced.csv")

    # Daily Node Costs
    df_daily_costs.to_csv('../data/daily_costs_june-aug.csv', index=False)
    print("Saved: daily_costs_june-aug.csv")

except Exception as e:
    print(f"Error while saving: {e}")

print("\nAnalysis fully completed!")
print("Results saved in /data/ folder")
print("Enhanced methodology applied for runtime calculation")
print("Separate pricing implemented: per-task for successful, per-second for failed tasks")

## Additional: Export Raw Task Data

We generate a detailed table for each task with key metrics:
- Task date
- Node type (A100 / L4)
- Service
- Success / failure
- Runtime duration (seconds)
- Allocated computation cost
- Weighted average cost

In [ ]:
# Create a detailed task-level table based on df_intervals
df_tasks_raw = df_intervals.copy()

# Function to enrich task data with cost info
def add_cost_info_to_tasks(df_tasks_raw, df_cost_per_gen_daily):
    tasks_with_cost = []

    for _, task_row in df_tasks_raw.iterrows():
        task_date = task_row['date']
        service_name = task_row['service_name']
        node_type = task_row['node_type']
        success = task_row['success']
        duration_seconds = task_row['duration_seconds']

        # Match corresponding cost entry
        cost_row = df_cost_per_gen_daily[
            (df_cost_per_gen_daily['date'] == task_date) &
            (df_cost_per_gen_daily['service_name'] == service_name) &
            (df_cost_per_gen_daily['node_type'] == node_type) &
            (df_cost_per_gen_daily['success'] == success)
        ]

        if len(cost_row) > 0:
            cost_info = cost_row.iloc[0]

            if success:
                # Successful tasks: cost per generation
                allocated_cost = cost_info['cost_per_generation']
                weighted_avg_cost = cost_info['cost_per_generation']
            else:
                # Failed tasks: cost per second × duration
                cost_per_second = cost_info['cost_per_second']
                allocated_cost = cost_per_second * duration_seconds if cost_per_second else 0
                weighted_avg_cost = cost_per_second

            tasks_with_cost.append({
                'task_id': task_row['task_id'],
                'task_date': task_date,
                'node_type': node_type,
                'service_name': service_name,
                'success': success,
                'duration_seconds': duration_seconds,
                'allocated_cost_per_task': allocated_cost,
                'weighted_avg_cost': weighted_avg_cost,
                'cost_type': 'per_task' if success else 'per_second'
            })

    return pd.DataFrame(tasks_with_cost)

# Generate the detailed table
df_tasks_detailed = add_cost_info_to_tasks(df_tasks_raw, df_cost_per_gen_daily)

print(f"Detailed task table created: {len(df_tasks_detailed)} rows")

if len(df_tasks_detailed) > 0:
    print("\n=== STRUCTURE OF DETAILED TABLE ===")
    display(df_tasks_detailed.head(10))

    print(f"\n=== STATS ===")
    print(f"Total tasks: {len(df_tasks_detailed):,}")
    print(f"Successful: {df_tasks_detailed['success'].sum():,}")
    print(f"Failed: {(~df_tasks_detailed['success']).sum():,}")
    print(f"Total allocated cost: ${df_tasks_detailed['allocated_cost_per_task'].sum():.2f}")

    # Save to CSV
    try:
        df_tasks_detailed.to_csv('../data/tasks_raw_detailed_enhanced.csv', index=False)
        print("\nSaved: tasks_raw_detailed_enhanced.csv")
    except Exception as e:
        print(f"\nError while saving: {e}")